In [3]:
import os
import time
import base64
from PIL import Image
from io import BytesIO
from selenium import webdriver
import chromedriver_autoinstaller
import logging
from selenium.webdriver.common.by import By  # Import the By class


In [4]:
# Setup logging
logging.basicConfig(filename='image_capture_log.txt', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Set up the directory where you want to save the images
save_directory = "/home/skierroe412/image_saving_from_Duckie_stream"
os.makedirs(save_directory, exist_ok=True)

# Setup WebDriver
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

url = "http://littlenedr4.local/dashboard/robot/mission_control"
driver.get(url)
time.sleep(15)  # Ensure enough time for all connections and scripts to load

try:
    while True:
        start_time = time.time()

        element = driver.find_element(By.ID, 'block_MnSpIzaJ')  # Updated to use By.ID
        style = element.value_of_css_property('background-image')
        
        # Check and process the base64 string
        if 'base64' in style:
            start = style.find("base64,") + 7  # Adjust index to cut 'base64,' part
            end = style.rfind('")')  # Adjust index for the ending quotes and parenthesis
            base64_str = style[start:end]
            image_data = base64.b64decode(base64_str)
            image = Image.open(BytesIO(image_data))
            filename = f"image_{int(time.time()*1000)}.jpeg"
            image_path = os.path.join(save_directory, filename)
            image.save(image_path)
            logging.info(f"Saved image at: {image_path}")

        # Adjust timing to repeat approximately every 100ms
        elapsed_time = time.time() - start_time
        if elapsed_time < 0.1:
            time.sleep(0.1 - elapsed_time)

except Exception as e:
    logging.error(f"Error occurred: {e}")

finally:
    driver.quit()